# SmartDoc Insight
### This project is designed to fetch data from a variety of sources, including PDF, text, URL and answer the users queries. It utilizes libraries such as Pandas, Vector Database(FAISS), Langchain and RAG concept.


# 1. Install the neccessary Libraries

In [1]:
!pip install langchain
!pip install -U langchain-community
!pip install -U unstructured
!pip install sentence-transformers
!pip install faiss-cpu
!pip install langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 59.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 4.

In [2]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pickle
import os
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain

# 2. Extract Text from URL,PDF,Text file

In [3]:

# Define URLs, PDF paths, and text file paths
sources = [
    r"https://www.livemint.com/market/stock-market-news/bhel-share-price-jumps-over-7-to-hit-a-52-week-high-up-over-60-this-year-so-far-should-you-buy-sell-or-hold-11714710174350.html",
    r"/content/the alchemist.pdf",
    r"/content/read.txt"
]

# Load data from different sources
data = []
for source in sources:
    if source.endswith('.pdf'):
        # Load data from PDF
        loader = PyPDFLoader(source)
        pdf_data = [loader.load()]
        data.extend(pdf_data)
    elif source.endswith('.txt'):
        # Load data from text file
        loader = TextLoader(source)
        text_data = loader.load()
        data.append(text_data)
    elif source.endswith('.html'):
        # Load data from URL
        loader = UnstructuredURLLoader(urls=[source])
        url_data = [loader.load()]
        data.extend(url_data)

print(len(data))
print(type(data))



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


3
<class 'list'>


In [4]:
data = [item for sublist in data for item in sublist]
data

[Document(page_content='Sign in\n\ne-paper\n\nSubscribe\n\nSign In\n\nHome\n\nTuesday, 18 June 2024\n\nStocks\n\nMutual Funds\n\nNews\n\nHome\n\nNews\n\nLatest News\n\nMarkets\n\nPremium\n\nMoney\n\nMutual Fund\n\nIndustry\n\nCompanies\n\nTechnology\n\nWeb Stories\n\nIn Charts\n\nOpinion\n\nVideos\n\nAll\n\nCompanies\n\nTechnology\n\nMarkets\n\nMoney\n\nMyMint\n\nMutual Funds\n\nInsurance\n\nAuto\n\nIndustry\n\nPersonal Finance\n\nHello User\n\nSign in\n\nSign Out\n\nMy Account\n\nMy Account\n\nSubscribe\n\nMy Watchlist\n\nNewsletters\n\nNotifications\n\nMy Reads\n\nFor You\n\nView Less -\n\nView More +\n\nData Insights\n\nMarket Dashboard\n\nBullion      Gold   Silver\n\nFuel      Petrol   Diesel\n\nCommodities      Gold   GoldM   Aluminum   Menthaoil   Silver   SilverMIC   GoldPetal   Natural Gas   Copper   Zinc   SilverM   CrudeOil   GoldGinuea   Lead\n\nCryptoCurrencies\n\nView Less -\n\nView More +\n\nTop Sections\n\nToday News      India News   World News\n\nEconomy\n\nCompany Bu

# 3. Create the Embeddings of Data fetched from different data sources.

In [5]:

embeddings=HuggingFaceEmbeddings()



/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# 4. Store the embeddings in Vectore Database

In [6]:
#from langchain.embeddings import HuggingFaceEmbeddings
# !pip install faiss-cpu
vector_index_combo=FAISS.from_documents(data,embeddings)

In [7]:
# storing vector index create in local
import pickle
file_path_1="vector_store_combo.pkl"
with open(file_path_1,"wb") as f:
    pickle.dump(vector_index_combo,f)


In [8]:
import os
file_path_1="vector_store_combo.pkl"
if os.path.exists(file_path_1):
    with open (file_path_1,"rb") as f:
        Vector_Index=pickle.load(f)

# Create

In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI

google_api_key = "Add_your_API_key"


llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest", google_api_key=google_api_key)


In [11]:
from langchain.chains import LLMChain
from langchain import PromptTemplate
from langchain.chains import RetrievalQA

In [13]:
retriever =  Vector_Index.as_retriever()
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    verbose=True
)

In [14]:
response=qa.run({"query": "who is Santiago.Explain is journey in brief."})
response

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.


"Santiago is the protagonist of Paulo Coelho's novel *The Alchemist*. He is a young shepherd from Andalusia who dreams of traveling the world. \n\nOne day, Santiago meets an old man who tells him about a treasure buried near the Pyramids of Egypt. Inspired by the encounter, Santiago decides to follow his dream and embarks on a journey to find the treasure. He travels across the Sahara Desert, faces various challenges, and meets influential characters who guide him on his path. Ultimately, Santiago's journey becomes about self-discovery and realizing that his true treasure, and his Personal Legend, was within him all along. \n"

In [15]:
response = response.strip()
response

"Santiago is the protagonist of Paulo Coelho's novel *The Alchemist*. He is a young shepherd from Andalusia who dreams of traveling the world. \n\nOne day, Santiago meets an old man who tells him about a treasure buried near the Pyramids of Egypt. Inspired by the encounter, Santiago decides to follow his dream and embarks on a journey to find the treasure. He travels across the Sahara Desert, faces various challenges, and meets influential characters who guide him on his path. Ultimately, Santiago's journey becomes about self-discovery and realizing that his true treasure, and his Personal Legend, was within him all along."

In [16]:
response=qa.run({"query": "Why BHEL's Prices are rising?"})
response



> Entering new RetrievalQA chain...

> Finished chain.


"BHEL's share price is rising because:\n\n* **Strategic Partnership:** They partnered with HIMA Middle East FZE for railway signalling, expanding their offerings to Indian Railways.\n* **Potential Large-Cap Status:** Analysts believe BHEL might be recategorised as a large-cap stock, attracting more investors.\n* **Positive Market Sentiment:** The stock has broken critical resistance levels, indicating bullishness and upward momentum.\n* **Strong Uptrend:**  BHEL has been consistently forming higher highs and higher lows, a sign of a robust uptrend.\n\n**Disclaimer:** I am an AI chatbot. Consult a financial advisor for investment decisions. \n"

In [17]:
response = response.strip()
response

"BHEL's share price is rising because:\n\n* **Strategic Partnership:** They partnered with HIMA Middle East FZE for railway signalling, expanding their offerings to Indian Railways.\n* **Potential Large-Cap Status:** Analysts believe BHEL might be recategorised as a large-cap stock, attracting more investors.\n* **Positive Market Sentiment:** The stock has broken critical resistance levels, indicating bullishness and upward momentum.\n* **Strong Uptrend:**  BHEL has been consistently forming higher highs and higher lows, a sign of a robust uptrend.\n\n**Disclaimer:** I am an AI chatbot. Consult a financial advisor for investment decisions."